In [2]:
import pandas as pd
import numpy as np

# Remplacer 'chemin_du_fichier' par le chemin réel du fichier CSV
df = pd.read_csv('C:/Users/33640/OneDrive/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

print(df.columns.tolist())

['ticker', 'open', 'high', 'low', 'close', 'volume', 'OPCL', 'pvCLCL', 'prevAdjClose', 'SPpvCLCL', 'sharesOut', 'PERMNO', 'SICCD', 'PERMCO', 'prevRawOpen', 'prevRawClose', 'prevAdjOpen']


In [3]:
import pandas as pd
import ast

# Fonction pour convertir en toute sécurité une chaîne de caractères en liste
def safe_literal_eval(s):
    try:
        # Tente de convertir la chaîne en liste
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # Si une erreur se produit, retourne une valeur par défaut, par exemple une liste vide
        return []


# Appliquer la fonction de conversion aux colonnes 'open' et 'close'
df['open'] = df['open'].apply(safe_literal_eval)
df['close'] = df['close'].apply(safe_literal_eval)

# Calculer les retours pour chaque ligne
df['return'] = df.apply(lambda row: [(c - o) / o for o, c in zip(row['open'], row['close'])], axis=1)

# Afficher les premières lignes pour vérifier
print(df.head())

  ticker  ...                                             return
0     AA  ...  [-0.012957317073170731, 0.010093167701863354, ...
1    ABM  ...  [-0.009146341463414634, 0.012422360248447204, ...
2    ABT  ...  [-0.0070921985815602835, -0.012704174228675136...
3    ADI  ...  [-0.03542780748663102, -0.04329608938547486, 0...
4    ADM  ...  [0.0, 0.005291005291005291, -0.015789473684210...

[5 rows x 18 columns]


In [4]:
new_df = df[['ticker', 'return']]

new_df.shape

(664, 2)

In [77]:
# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

def check_nan_inf(df):
    # Vérification des valeurs NaN
    if df.isna().any().any():
        print("Il y a des valeurs NaN dans le DataFrame")
    else:
        print("Il n'y a pas de valeurs NaN dans le DataFrame")

def remove_rows_with_nan(df):
    return df.dropna()

df_cleaned = remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

check_nan_inf(df_cleaned)

df_cleaned.shape

Il n'y a pas de valeurs NaN dans le DataFrame


(632, 5532)

In [8]:
for i in range(20):
    start_col = i * 250 + 1  # Commence à partir de la deuxième colonne
    end_col = start_col + 250

    # Vérifie si la fin de la tranche dépasse le nombre de colonnes
    if end_col > len(df_cleaned.columns):
        end_col = len(df_cleaned.columns)  # Ajuste pour ne pas dépasser

data_period = df_cleaned.iloc[:, start_col:end_col]

data_period.head()


,return_4750,return_4751,return_4752,return_4753,return_4754,return_4755,return_4756,return_4757,return_4758,return_4759,return_4760,return_4761,return_4762,return_4763,return_4764,return_4765,return_4766,return_4767,return_4768,return_4769,return_4770,return_4771,return_4772,return_4773,return_4774,return_4775,return_4776,return_4777,return_4778,return_4779,return_4780,return_4781,return_4782,return_4783,return_4784,return_4785,return_4786,return_4787,return_4788,return_4789,...,return_4960,return_4961,return_4962,return_4963,return_4964,return_4965,return_4966,return_4967,return_4968,return_4969,return_4970,return_4971,return_4972,return_4973,return_4974,return_4975,return_4976,return_4977,return_4978,return_4979,return_4980,return_4981,return_4982,return_4983,return_4984,return_4985,return_4986,return_4987,return_4988,return_4989,return_4990,return_4991,return_4992,return_4993,return_4994,return_4995,return_4996,return_4997,return_4998,return_4999
0,0.008112,0.005247,-0.029558,0.071770,-0.073543,0.024807,-0.025836,-0.053254,-0.044113,-0.011173,-0.025034,0.012115,-0.005808,0.003508,-0.013684,0.004267,-0.025071,-0.011931,-0.034394,-0.042634,0.072303,0.028398,-0.018088,-0.006355,0.010981,0.004594,0.055887,-0.003856,-0.016707,0.017483,0.010108,-0.005919,-0.009291,-0.001382,-0.007194,0.048919,-0.021725,-0.041034,0.005329,-0.006750,...,-0.022296,-0.022406,-0.016345,-0.006179,-0.004182,0.018325,-0.008243,-0.021693,-0.018133,0.002682,0.006263,0.007886,0.018315,-0.004675,-0.012165,0.042594,-0.050997,0.013692,0.022585,-0.028116,0.016441,0.014138,0.006993,-0.000469,-0.007163,0.039943,0.028276,-0.026293,-0.022758,0.007143,0.006702,0.003160,-0.006717,-0.014399,-0.002850,-0.005238,0.006790,-0.021874,-0.001471,-0.013705
1,0.003652,-0.002294,0.000995,0.010880,0.021526,0.015385,-0.032439,-0.043240,-0.033585,-0.023379,-0.029813,-0.004378,-0.038377,-0.005376,-0.000770,0.031250,0.161509,0.021032,-0.027384,-0.037458,0.067995,0.004237,0.001611,0.024569,-0.007299,-0.008648,0.023011,0.042341,0.027917,-0.007673,0.008888,0.005279,0.003235,0.015878,0.018049,0.003156,0.023322,-0.015922,-0.020402,-0.019913,...,-0.014901,-0.002472,-0.019973,-0.032670,0.006195,0.016657,-0.003465,0.004981,-0.006071,0.016555,0.016320,0.016448,-0.007574,0.007393,0.001082,0.018740,-0.006329,0.008774,-0.015802,-0.016293,0.017964,0.006674,-0.005059,-0.007235,-0.009508,-0.007351,-0.001086,0.010831,-0.002952,-0.002931,0.010963,0.002928,0.015103,0.001318,0.009752,-0.017885,0.015071,0.006753,0.003378,-0.022182
2,0.002502,0.001164,0.014554,0.038944,0.011459,0.001623,-0.000269,-0.036907,-0.025087,0.012564,0.002512,-0.003846,-0.007289,-0.007852,-0.008951,-0.010868,-0.019050,0.000585,-0.011753,-0.018857,0.056449,0.032451,0.000563,0.010619,-0.012644,-0.036660,0.014750,0.013785,-0.017973,0.007399,0.005824,0.010641,-0.002046,0.030113,-0.009645,0.009303,0.006057,0.007327,-0.008369,0.012286,...,-0.014451,0.018007,-0.026062,-0.022973,0.022965,0.008487,-0.004423,-0.025447,0.002396,0.013276,-0.013626,0.001631,0.015861,0.005162,0.010550,0.004611,-0.017922,-0.015524,0.007188,-0.002819,0.002455,0.010513,0.012129,0.008281,-0.001910,-0.017006,-0.009875,-0.006396,0.007515,0.003128,0.004920,0.005402,0.008128,0.003097,0.000357,0.012283,-0.016918,-0.003900,-0.005336,-0.000478
3,-0.000113,-0.002658,0.001341,0.014425,-0.005112,0.024749,0.001821,-0.028989,-0.029752,0.024548,-0.014125,-0.015908,-0.013494,-0.002401,0.000688,-0.002040,-0.035051,-0.006007,-0.013068,-0.020582,0.038798,0.025222,-0.000351,-0.003020,0.022492,-0.027232,0.015099,0.008117,0.011511,0.018411,0.019795,0.022596,-0.013306,-0.000570,-0.006849,0.026587,0.019529,-0.015899,-0.006040,0.046614,...,-0.016997,-0.003745,-0.043812,-0.001466,0.018371,0.010550,-0.018420,-0.035094,0.002784,0.005937,0.013579,-0.002080,0.022935,-0.018809,-0.015836,-0.006263,0.008349,-0.003326,-0.017953,-0.009871,0.012127,0.010492,-0.010802,-0.020451,-0.009935,0.016261,0.002881,0.017763,-0.008584,-0.013009,0.009287,0.000000,0.002035,0.017014,-0.003342,-0.011334,-0.0093

In [9]:
# Supprimez la colonne 'ticker' pour ne garder que les données numériques
returns_only = data_period

returns_transposed = returns_only.transpose()

# Calculer la matrice de corrélation sur les actifs transposés
correlation_matrix = returns_transposed.corr(method='pearson')


# Afficher la matrice de corrélation
print(correlation_matrix)

          0         1         2    ...       661       662       663
0    1.000000  0.299607  0.327069  ...  0.342843  0.088565  0.136778
1    0.299607  1.000000  0.291538  ...  0.169579  0.156215  0.081944
2    0.327069  0.291538  1.000000  ...  0.295753  0.409285  0.171438
3    0.382164  0.223630  0.460261  ...  0.355419  0.194597  0.130567
4    0.333826  0.226859  0.337487  ...  0.425380  0.153418  0.201059
..        ...       ...       ...  ...       ...       ...       ...
659  0.557932  0.394316  0.591948  ...  0.527833  0.438272  0.353846
660  0.403585  0.121729  0.349916  ...  0.387907  0.204485  0.305091
661  0.342843  0.169579  0.295753  ...  1.000000  0.213686  0.296405
662  0.088565  0.156215  0.409285  ...  0.213686  1.000000  0.231740
663  0.136778  0.081944  0.171438  ...  0.296405  0.231740  1.000000

[632 rows x 632 columns]


In [10]:
def signed_adjency(mat):
    '''
    L'idée est ici, à partir d'une matrice de corrélation mat, de renvoyer deux matrices 
    A_positive et A_negative qui correspondraient aux matrices des corrélations positives et négatives 
    associées  
    '''

    A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
    A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
    
    return A_pos, A_neg

signed_adjency(correlation_matrix)

(          0         1         2    ...       661       662       663
 0    1.000000  0.299607  0.327069  ...  0.342843  0.088565  0.136778
 1    0.299607  1.000000  0.291538  ...  0.169579  0.156215  0.081944
 2    0.327069  0.291538  1.000000  ...  0.295753  0.409285  0.171438
 3    0.382164  0.223630  0.460261  ...  0.355419  0.194597  0.130567
 4    0.333826  0.226859  0.337487  ...  0.425380  0.153418  0.201059
 ..        ...       ...       ...  ...       ...       ...       ...
 659  0.557932  0.394316  0.591948  ...  0.527833  0.438272  0.353846
 660  0.403585  0.121729  0.349916  ...  0.387907  0.204485  0.305091
 661  0.342843  0.169579  0.295753  ...  1.000000  0.213686  0.296405
 662  0.088565  0.156215  0.409285  ...  0.213686  1.000000  0.231740
 663  0.136778  0.081944  0.171438  ...  0.296405  0.231740  1.000000
 
 [632 rows x 632 columns],
      0    1    2    3    4    5    6    ...  657  658  659  660  661  662  663
 0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  

In [11]:
import sys

sys.path.append('C:/Users/33640/OneDrive/Documents/GitHub/Portfolio_clustering_project')  # Ajoute le chemin parent

from signet.cluster import Cluster 

In [12]:
from scipy import sparse

def apply_SPONGE(correlation_matrix, k): 

    '''
    IDÉE : étant donné une matrice de correlation obtenue à partir d'une base de donnée et de la similarité de pearson, renvoyer un vecteur associant 
           à chaque actif le numéro du cluster auquel il appartient une fois qu'on lui a appliqué SPONGE (à partir du package signet)

    PARAMS : 

    - correlation_matrix : a square dataframe of size (number_of_stocks, number_of_stocks)
    - k : the number of clusters to identify. If a list is given, the output is a corresponding list

    RETURNS : array of int, or list of array of int: Output assignment to clusters.

    '''
    
    ## On respecte le format imposé par signet. Pour cela il faut changer le type des matrices A_pos et A_neg, qui ne peuvent pas rester des dataframes 

    A_pos, A_neg = signed_adjency(correlation_matrix)

    A_pos_sparse = sparse.csc_matrix(A_pos.values)
    A_neg_sparse = sparse.csc_matrix(A_neg.values)

    data = (A_pos_sparse, A_neg_sparse)

    cluster = Cluster(data)

    ## On applique la méthode SPONGE : clusters the graph using the Signed Positive Over Negative Generalised Eigenproblem (SPONGE) clustering.

    return cluster.SPONGE(k )

In [76]:
result = apply_SPONGE(correlation_matrix, k=30)

print(len(result))


632


In [15]:
l=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in result:
    for j in range(30):
        if i==j:
            l[i]+=1

print(l)

[9, 4, 145, 4, 8, 322, 4, 44, 3, 1, 2, 1, 1, 1, 5, 1, 36, 4, 1, 22, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0]


In [35]:
data_period=[]
result=[]
for i in range(28):
    start_col = i * 200 + 1  # Commence à partir de la deuxième colonne
    end_col = start_col + 200

    # Vérifie si la fin de la tranche dépasse le nombre de colonnes
    if end_col > len(df_cleaned.columns):
        end_col = len(df_cleaned.columns)  # Ajuste pour ne pas dépasser
    
    data_period.append(df_cleaned.iloc[:, start_col:end_col]) 

    returns_transposed = data_period[i].transpose()

    # Calculer la matrice de corrélation sur les actifs transposés
    correlation_matrix = returns_transposed.corr(method='pearson')
    result.append(apply_SPONGE(correlation_matrix, k=9))



In [36]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# Supposons que vous avez une liste de listes, où chaque sous-liste est un des 28 clusterings
# Chaque sous-liste contient 632 éléments avec des valeurs de 0 à 9
clusterings = result  # Remplacez ceci par vos données de clustering

# Convertir les résultats de clustering en une matrice de co-occurrence
n_data = 632
co_occurrence_matrix = np.zeros((n_data, n_data))

for clustering in clusterings:
    for i in range(n_data):
        for j in range(n_data):
            if clustering[i] == clustering[j]:
                co_occurrence_matrix[i, j] += 1

# Convertir en matrice de similarité
similarity_matrix = co_occurrence_matrix / len(clusterings)

# Convertir la matrice de similarité en matrice de distance
distance_matrix = 1 - similarity_matrix

# Convertir la matrice de distance en format condensé pour la fonction linkage
# car linkage attend une matrice de distance condensée pour les données non euclidiennes
tri_upper_idx = np.triu_indices(n_data, k=1)
distance_condensed = distance_matrix[tri_upper_idx]

# Effectuer le clustering hiérarchique
Z = linkage(squareform(distance_condensed), method='average')

# Former des clusters
# Vous pouvez ajuster le 't' pour obtenir le nombre de clusters souhaité
clusters = fcluster(Z, t=9, criterion='maxclust')

# Afficher les clusters
print(clusters)

[4 4 6 4 4 4 9 4 3 9 7 4 4 4 4 4 8 4 4 4 4 4 4 4 4 7 4 4 4 5 4 4 8 4 3 4 4
 4 9 4 3 9 8 4 4 2 7 4 4 4 4 4 4 4 6 4 4 4 8 4 4 4 8 4 4 4 4 9 1 1 4 4 4 6
 4 3 4 7 4 4 4 4 4 8 4 6 4 4 4 4 3 4 6 4 4 4 4 4 2 2 6 5 4 4 6 4 4 9 1 4 4
 5 4 4 4 6 5 9 8 4 4 5 4 7 4 4 4 8 4 4 7 4 1 9 4 4 7 4 4 2 4 4 2 4 4 2 4 4
 8 4 5 1 2 9 9 5 4 4 4 4 9 4 8 9 4 4 4 4 4 4 5 5 8 8 5 4 8 4 9 2 4 4 4 4 4
 4 4 4 3 4 7 4 7 4 4 9 4 4 4 4 4 4 8 8 4 2 7 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2
 6 4 4 4 4 4 4 2 4 4 4 5 4 8 4 9 4 4 2 8 2 3 4 4 4 4 5 4 4 8 4 6 4 6 4 4 4
 2 4 9 4 4 4 4 4 4 5 5 4 4 1 7 4 4 4 4 4 6 4 4 4 4 4 6 4 4 4 4 8 6 4 4 6 4
 7 8 1 4 1 7 4 4 1 5 4 4 6 4 4 4 9 4 4 8 3 4 8 4 8 8 4 4 4 4 4 1 4 4 2 4 4
 4 6 7 4 1 1 4 1 1 4 1 1 1 6 4 4 4 4 2 1 6 4 1 1 1 6 5 2 4 4 4 4 4 4 4 4 1
 1 1 5 1 1 4 1 1 4 1 1 1 1 1 5 4 1 4 3 5 8 9 9 4 4 8 8 4 4 4 5 4 2 4 4 1 4
 4 4 1 4 8 4 4 4 9 8 4 1 5 5 4 4 4 4 5 4 2 9 9 8 5 6 2 6 2 6 4 4 4 4 4 4 2
 4 4 4 8 1 1 4 9 4 9 4 9 2 8 8 4 4 5 4 4 4 2 4 4 8 4 4 4 4 5 4 4 4 4 4 4 4
 4 4 4 5 4 4 4 4 4 4 7 5 

C:\Users\33640\AppData\Local\Temp/ipykernel_4388/2403182135.py:31: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(squareform(distance_condensed), method='average')


In [37]:
l=[0,0,0,0,0,0,0,0,0,0,0]
for i in clusters:
    for j in range(1,11):
        if i==j:
            l[i]+=1

print(l)

[0, 42, 27, 11, 395, 39, 27, 16, 43, 32, 0]


Now we want to run markowitz on our portfolio, we need to get the returns and the covariance matrix of the returns of each asset, but before we need to get the composition of each cluster which is a combination of multiple assets. The we get the means of the returns of each cluster, and the covariance of the returns of the clusters.

In [78]:
cluster_comp=[[],[],[],[],[],[],[],[],[]]
cluster_comp2=[]
for c in range(632):
    cluster_comp[clusters[c]-1].append(df_cleaned.iloc[c, 0])
for cluster in cluster_comp:
    cluster_indices = [df_cleaned.index[df_cleaned['ticker'] == element].tolist() for element in cluster]
    cluster_comp2.append([index for sublist in cluster_indices for index in sublist])



print(cluster_comp2[3])

[0, 1, 3, 4, 5, 7, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59, 60, 61, 63, 64, 65, 66, 70, 71, 72, 74, 76, 78, 79, 80, 81, 82, 84, 86, 87, 88, 89, 91, 93, 94, 95, 96, 97, 102, 103, 105, 106, 109, 110, 112, 113, 114, 119, 120, 122, 124, 125, 126, 128, 129, 131, 134, 135, 137, 138, 140, 141, 143, 144, 146, 147, 149, 156, 157, 158, 159, 161, 164, 165, 166, 167, 168, 169, 175, 177, 180, 181, 182, 183, 184, 185, 186, 187, 189, 191, 193, 194, 196, 197, 198, 199, 200, 201, 204, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 223, 224, 225, 226, 227, 228, 230, 231, 232, 234, 236, 238, 239, 244, 245, 246, 247, 249, 250, 252, 254, 256, 257, 258, 260, 262, 263, 264, 265, 266, 267, 270, 271, 274, 275, 276, 277, 278, 280, 281, 282, 283, 284, 286, 287, 288, 289, 292, 293, 295, 299, 302, 303, 306, 307, 309, 310, 311, 313, 314, 317, 319, 322, 323, 324, 325, 326, 328, 329, 331, 332,

In [79]:
mean_returns = df_cleaned.mean(axis=1).tolist()
clusters_mean_return=[0,0,0,0,0,0,0,0,0]
for c in range (632):
    clusters_mean_return[clusters[c]-1]+=mean_returns[c]/len(cluster_comp[clusters[c]-1])

print(clusters_mean_return)
annual_expected_returns=[]
for c in range(len(clusters_mean_return)):
    annual_expected_returns.append((1+clusters_mean_return[c])**365-1)

print(annual_expected_returns)

[4.632719063325762e-05, 0.00015003423920513514, -0.00035910479058915963, 0.0003150790204265111, -3.648594873043111e-06, 0.00041101050666552617, 0.00024126980441708655, 0.000462897403780838, 0.0003796517243677526]
[0.017052799783644268, 0.05628537430626035, -0.12286712628269869, 0.12185742699441704, -0.00133085318652848, 0.16182031674022168, 0.09204584254223125, 0.18402358903803928, 0.1486031837923376]


C:\Users\33640\AppData\Local\Temp/ipykernel_4388/3045893782.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_returns = df_cleaned.mean(axis=1).tolist()


In [85]:
return_clusters = pd.DataFrame(columns=df_cleaned.columns[1:])

# Remplissage du nouveau DataFrame avec les moyennes des colonnes pour chaque cluster
for k in range(9):
    # Sélection des lignes appartenant au cluster k
    rows_in_cluster = df_cleaned.iloc[cluster_comp2[k]]

    # Calcul de la moyenne pour chaque colonne, en excluant la première colonne
    mean_values = rows_in_cluster.iloc[:, 1:].mean()

    # Ajout des moyennes calculées au nouveau DataFrame
    return_clusters.loc[k] = mean_values

returns_transposed = return_clusters.transpose()

# Calculer la matrice de corrélation sur les actifs transposés
correlation_matrix = returns_transposed.cov()


# Afficher la matrice de corrélation
print(correlation_matrix)

          0         1         2  ...         6         7         8
0  0.000025  0.000020  0.000015  ...  0.000012  0.000019  0.000012
1  0.000020  0.000043  0.000033  ...  0.000031  0.000048  0.000025
2  0.000015  0.000033  0.000244  ...  0.000059  0.000082  0.000048
3  0.000014  0.000040  0.000077  ...  0.000089  0.000132  0.000065
4  0.000018  0.000045  0.000132  ...  0.000091  0.000120  0.000070
5  0.000007  0.000016  0.000029  ...  0.000046  0.000057  0.000050
6  0.000012  0.000031  0.000059  ...  0.000095  0.000097  0.000064
7  0.000019  0.000048  0.000082  ...  0.000097  0.000247  0.000083
8  0.000012  0.000025  0.000048  ...  0.000064  0.000083  0.000104

[9 rows x 9 columns]


In [87]:
from module1 import markowitz

markowitz(annual_expected_returns, correlation_matrix)

OrderedDict([(0, 0.0),
             (1, 0.0),
             (2, 0.0),
             (3, 0.0),
             (4, 0.0),
             (5, 0.91591),
             (6, 0.0),
             (7, 0.05419),
             (8, 0.0299)])